In [51]:
import cv2 as cv
import matplotlib.pyplot as plt
import math
import numpy as np
import random
import pandas as pd
import pickle
import os

import keras.utils
from keras import backend as K

from keras.applications import mobilenet, resnet50, inception_resnet_v2, inception_v3, vgg16
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import dot, Dropout, Dense, GlobalAveragePooling2D, Input, Lambda
from keras.models import Model
from keras.optimizers import RMSprop
from keras.regularizers import l2

## Load Training / Validation Encodings

In [52]:
with open('x_train_pairs_resnet.pickle', 'rb') as f:
    x_train_encoding = pickle.load(file=f)

with open('x_val_pairs_resnet.pickle', 'rb') as f:
    x_val_encoding = pickle.load(file=f)

print("x_train encodings: " + str(x_train_encoding.shape))
print("x_val encodings: " + str(x_val_encoding.shape))

x_train encodings: (122400, 2, 2048)
x_val encodings: (13600, 2, 2048)


In [53]:
def create_labels(n_labels):
    
    y = []
    
    for i in range(0,n_labels):
        if i%2==0: 
            y.append(1)
        else:
            y.append(0)
            
    return np.array(y, np.int)

y_train = create_labels(x_train_encoding.shape[0])
y_val = create_labels(x_val_encoding.shape[0])
print("y_train: "+str(y_train.shape))
print("y_val: "+str(y_val.shape))

y_train: (122400,)
y_val: (13600,)


## Define Siamese Model

In [57]:
# calculate cosine distance b/t feature vector outputs from base network
def cos_distance(feat_vects):

    K.set_epsilon(1e-2)
    epsilon = K.epsilon()

    x1, x2 = feat_vects

    result = K.maximum(x=dot(inputs=[x1, x2], axes=1, normalize=True), y=epsilon)

    return result
 
# calculate l1_norm b/t feature vector outputs from base network
def l1_distance(feat_vects):
    
    K.set_epsilon(1e-07)
    epsilon = K.epsilon()

    x1, x2 = feat_vects

    result = K.maximum(x=K.sum(x=K.abs(x1-x2), axis=1, keepdims=True), y=epsilon)

    return result
 

# calculate l2_distance b/t feature vector outputs from base network
def l2_distance(feat_vects):
    
    K.set_epsilon(1e-07)
    epsilon = K.epsilon()

    x1, x2 = feat_vects

    result = K.sqrt(K.maximum(x=K.sum(x=K.square(x1 - x2), axis=1, keepdims=True), y=epsilon))

    return result


# create a siamese model that calculates similarity b/t two feature vectors
def create_siamese_model(encoding_shape, similarity_metric):

    encoding_a = Input(shape=encoding_shape, name='encoding_a')
    encoding_b = Input(shape=encoding_shape, name='encoding_b')
    
    drop_a = Dropout(rate=0.6)(encoding_a)
    drop_b = Dropout(rate=0.6)(encoding_b)

    fc1_a = Dense(units=4096, activation='relu', kernel_regularizer=l2(l=0.0000), name='fc1_a')(drop_a)
    fc1_b = Dense(units=4096, activation='relu', kernel_regularizer=l2(l=0.0000), name='fc1_b')(drop_b)

    # fc1_a = Dropout(rate=0.3)(fc1_a)
    # fc1_b = Dropout(rate=0.3)(fc1_b)

    if similarity_metric == 'cosine':
        distance = Lambda(function=cos_distance, name='cos_distance')([fc1_a, fc1_b])
      
    elif similarity_metric == 'l1':
        distance = Lambda(function=l1_distance, name='l1_distance')([fc1_a, fc1_b])
      
    elif similarity_metric == 'l2':
        distance = Lambda(function=l2_distance, name='l2_distance')([fc1_a, fc1_b])

    prediction = Dense(units=1, activation='sigmoid', kernel_regularizer=l2(l=0.0000), name='sigmoid')(distance)

    model = Model(inputs=[encoding_a, encoding_b], outputs=prediction, name='siamese_model')

    return model

# create siamese model
encoding_shape = x_train_encoding.shape[2:]
print(encoding_shape)

cosine_model = create_siamese_model(encoding_shape, 'cosine')
print(cosine_model.summary())

l1_model = create_siamese_model(encoding_shape, 'l1')
# print(l1_model.summary())

l2_model = create_siamese_model(encoding_shape, 'l2')
# print(l2_model.summary())

(2048,)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoding_a (InputLayer)         (None, 2048)         0                                            
__________________________________________________________________________________________________
encoding_b (InputLayer)         (None, 2048)         0                                            
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 2048)         0           encoding_a[0][0]                 
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 2048)         0           encoding_b[0][0]                 
__________________________________________________________________________________________________
fc

## Training

In [58]:
def step_decay_schedule(lr_initial=0.001, decay=0.75, step_size=10):
    def schedule(epoch):
        return lr_initial * math.pow(decay, math.floor((1 + epoch) / step_size))

    return LearningRateScheduler(schedule=schedule, verbose=1)


def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr

    return lr


def create_callbacks(lr_type, wts_fn, enable_early_stopping=False, enable_save_wts=False):
    cbks = []

    # learning rate
    if lr_type is 0:
        lr_schedule = step_decay_schedule()
        cbks.append(lr_schedule)

    elif lr_type is 1:
        reduce_lr_schedule = ReduceLROnPlateau(monitor='val_loss',
                                               factor=0.1,
                                               patience=5,
                                               min_lr=1e-6,
                                               verbose=1)
        cbks.append(reduce_lr_schedule)

    # early stopping
    if enable_early_stopping is True:
        early_stopper = EarlyStopping(monitor='val_loss', patience=10)
        cbks.append(early_stopper)

    # model checkpoint
    if enable_save_wts is True:
        model_chpt = ModelCheckpoint(filepath=wts_fn,
                                     monitor='val_loss',
                                     verbose=1,
                                     save_weights_only=True,
                                     save_best_only=False,
                                     period=10)

        cbks.append(model_chpt)

    return cbks


### Cosine

In [61]:
# create callbacks
lr_type = 3  # 0=step decay, 1=val_loss decay
cosine_cbks = create_callbacks(lr_type, 'traingen_wts13.h5', True, False)

# training setup
batch_size = 32
n_epochs = 5
optim = RMSprop(lr=1e-3)
lr_metric = get_lr_metric(optim)

cosine_model.compile(loss="binary_crossentropy", optimizer=optim, metrics=['accuracy',lr_metric])
# K.clear_session()

print(x_train_encoding[:, 1].shape)
hist_cosine = cosine_model.fit(x=[x_train_encoding[:, 0], x_train_encoding[:, 1]],
                               y=y_train,
                               batch_size=batch_size,
                               epochs=n_epochs,
                               validation_data=([x_val_encoding[:, 0], x_val_encoding[:, 1]], y_val),
                               shuffle=True,
                               verbose=1,
                               callbacks=cosine_cbks)

(122400, 2048)
Train on 122400 samples, validate on 13600 samples
Epoch 1/5
122400/122400 [==============================] - 73s 598us/step - loss: 0.2690 - acc: 0.8933 - lr: 0.0010 - val_loss: 0.3614 - val_acc: 0.8468 - val_lr: 0.0010
Epoch 2/5
122400/122400 [==============================] - 72s 591us/step - loss: 0.2666 - acc: 0.8941 - lr: 0.0010 - val_loss: 0.3629 - val_acc: 0.8495 - val_lr: 0.0010
Epoch 3/5
122400/122400 [==============================] - 72s 591us/step - loss: 0.2653 - acc: 0.8955 - lr: 0.0010 - val_loss: 0.3659 - val_acc: 0.8550 - val_lr: 0.0010
Epoch 4/5
122400/122400 [==============================] - 72s 592us/step - loss: 0.2623 - acc: 0.8978 - lr: 0.0010 - val_loss: 0.3749 - val_acc: 0.8507 - val_lr: 0.0010
Epoch 5/5
122400/122400 [==============================] - 72s 591us/step - loss: 0.2640 - acc: 0.8963 - lr: 0.0010 - val_loss: 0.3651 - val_acc: 0.8498 - val_lr: 0.0010


In [60]:
cosine_model.save_weights('traingen_wts13.h5')

### L1 Norm

In [ ]:
# create callbacks
lr_type = 3  # 0=step decay, 1=val_loss decay
l1_cbks = create_callbacks(lr_type, 'traingen_wts8.h5', False, True)

# training setup
batch_size = 32
n_epochs = 20
optim = RMSprop(lr=1e-2)
lr_metric = get_lr_metric(optim)

l1_model.compile(loss="binary_crossentropy", optimizer=optim, metrics=['accuracy',lr_metric])
# K.clear_session()

print(x_train_encoding[:, 1].shape)
hist_l1 = l1_model.fit(x=[x_train_encoding[:, 0], x_train_encoding[:, 1]],
                          y=y_train,
                          batch_size=batch_size,
                          epochs=n_epochs,
                          validation_data=([x_val_encoding[:, 0], x_val_encoding[:, 1]], y_val),
                          shuffle=True,
                          verbose=1,
                          callbacks=l1_cbks)

### L2 Norm

In [ ]:
# create callbacks
lr_type = 3  # 0=step decay, 1=val_loss decay
l2_cbks = create_callbacks(lr_type, 'traingen_wts9.h5', False, True)

# training setup
batch_size = 32
n_epochs = 20
optim = RMSprop(lr=1e-2)
lr_metric = get_lr_metric(optim)

l2_model.compile(loss="binary_crossentropy", optimizer=optim, metrics=['accuracy',lr_metric])
# K.clear_session()

print(x_train_encoding[:, 1].shape)
hist_l2 = l2_model.fit(x=[x_train_encoding[:, 0], x_train_encoding[:, 1]],
                          y=y_train,
                          batch_size=batch_size,
                          epochs=n_epochs,
                          validation_data=([x_val_encoding[:, 0], x_val_encoding[:, 1]], y_val),
                          shuffle=True,
                          verbose=1,
                          callbacks=l2_cbks)

## Plot Training Metrics

In [ ]:
def plot_training_metrics(title, x_axis_label, y_axis_label, y, x=None):
    
    if x is None:
        plt.plot(y[0])
        plt.plot(y[1])
    else:
        plt.plot(x,y[0])
        plt.plot(x,y[1])
    
    plt.title(title)
    plt.ylabel(y_axis_label)
    plt.xlabel(x_axis_label)
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()

In [ ]:
# Loss vs. epochs - Cosine
plot_training_metrics(title='Model Loss',x_axis_label='Epoch',y_axis_label='Loss',
                     y=[hist_cosine.history['loss'], hist_cosine.history['val_loss']])

In [ ]:
# Loss vs. epochs - L1 Norm
plot_training_metrics(title='Model Loss',x_axis_label='Epoch',y_axis_label='Loss',
                     y=[hist_l1.history['loss'], hist_l1.history['val_loss']])

In [ ]:
# Loss vs. epochs - L2 Norm
plot_training_metrics(title='Model Loss',x_axis_label='Epoch',y_axis_label='Loss',
                     y=[hist_l2.history['loss'], hist_l2.history['val_loss']])